# Парсер HeadHunter

### Импорт библиотек

In [ ]:
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json
from bs4 import BeautifulSoup

from datetime import datetime, timedelta
import time
import os
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets

### Делаем выгрузку из HeadHunter
Ссылка на API HH: https://github.com/hhru/api \

Создадим необходимые константы

In [ ]:
vacancies_df = pd.DataFrame()
base_url = "https://api.hh.ru/"

text = f'NAME:({input()})'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=7)
date_to = dt_now.isoformat()
date_from = dt_from.isoformat()

req_url = base_url + f'vacancies?text={text}&date_from={date_from}&date_to={date_to}&responses_count_enabled=True&per_page=100'

В выдаче API HH нет опыта работы, поэтому мы укажем его в запросе самостоятельно:\
Можно выбрать подходящий. 

In [ ]:
experiences = ['noExperience',
               'between1And3',
               'between3And6', 
               'moreThan6']

#### Напишем цикл для направления запросов к API (в 3 этапа: get, json и перебор страниц)

In [ ]:
for exp in experiences:
  page = 0
  url = req_url + f'&experience={exp}'
  try:
    r = requests.get(url + f'&page={page}')
    data = json.loads(r.text)
    print(exp, '-', data['pages'])
    items = data['items']

    for page in range(1, data['pages']): 
      r = requests.get(url + f'&page={page}')
      data = json.loads(r.text)
      items += data['items']  
    df = pd.DataFrame(items)
    df['experience'] = exp 
    vacancies_df = pd.concat([vacancies_df, df], ignore_index=True)
  except Exception as e: 
    print(exp, '-', e)
    time.sleep(10)

In [ ]:
vacancies_df.head(5)

In [ ]:
vacancies_df.info()

### Вытащим ценную информацию из словарей

Напишем функцию

In [ ]:
def return_id(x, key = 'id', nan_value = np.nan):
  try:
    return x[key]
  except Exception as e:
    return nan_value

Вытащим желаемые данные. Применим нашу функцию с помощью apply

In [ ]:
vacancies_df['employer_name'] = vacancies_df['employer'].apply(return_id, 
                                                               key = 'name')
vacancies_df['department_name'] = vacancies_df['department'].apply(return_id, 
                                                               key = 'name')
vacancies_df['salary_from'] = vacancies_df['salary'].apply(return_id, key = 'from') 
vacancies_df['salary_to'] = vacancies_df['salary'].apply(return_id, key = 'to') 
vacancies_df['salary_gross'] = vacancies_df['salary'].apply(return_id, key = 'gross') 
vacancies_df['salary_currency'] = vacancies_df['salary'].apply(return_id, key = 'currency')
vacancies_df['responses'] = vacancies_df['counters'].apply(return_id, key = 'responses')
vacancies_df['city'] = vacancies_df['area'].apply(return_id, key = 'name')

In [ ]:
vacancies_df.info()

### Посмотрим компании, предлагающие вакансии, а также города

In [ ]:
vacancies_df['employer_name'].value_counts().head(10).plot(kind='bar')

In [ ]:
vacancies_df['city'].value_counts().head(10).plot(kind='bar')

### Что с зарплатой?

In [ ]:
vacancies_df['salary_from'].isna().value_counts(normalize=True)

In [ ]:
vacancies_df['salary_to'].isna().value_counts(normalize=True)

In [ ]:
vacancies_df['salary_currency'].value_counts(normalize=True)

In [ ]:
salaries = vacancies_df.query("salary_currency == 'RUR' and experience in ['noExperience','between1And3']").groupby(
    ['employer_name']).mean()['salary_to'].reset_index()

In [ ]:
salaries_top = salaries.sort_values(by='salary_to', ascending=False, na_position='last')
salaries_top.head(10)

Визуализируем с помощью seaborn

In [ ]:
sns.set(style='whitegrid')
fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(10,7))          
title = 'График зарплат (от)'
fig.suptitle(title)
plt.xticks(rotation=90)
sns.barplot(data=salaries_top.head(10).sort_values(by='salary_to', ascending=False), x='employer_name', y='salary_to', color='lightblue', ax=ax)

### Давайте поищем вакансии

Сделаем срез по городам, например.

Дадим пользователю право выбрать нужный ему город

In [ ]:
city_vacs = vacancies_df[vacancies_df['city'] == input()]

### Посмотрим содержание вакансий

Сделаем срез вакансий по интересному нам опыту

In [ ]:
vacs_df = city_vacs.copy().reset_index(drop=True).query("experience in ['noExperience', 'between1And3']")

In [ ]:
vacs_df.head(10)

Выведем адреса

In [ ]:
vacs_df['url'].head()

С помощью библиотек json и requests получим нужную информацию, добавим магии и обобразим HTML

In [ ]:
HTML(json.loads(requests.get('https://api.hh.ru/vacancies/66920100?host=hh.ru').text)['description'])

Сохраним информацию в читаемом виде. С помощью библиотеки BeautifulSoup извлечем данные из HTML

In [ ]:
vacs = vacs_df['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [ ]:
vacs = vacs_df['url'].apply(lambda x: BeautifulSoup(json.loads(requests.get(x).text)['description']).get_text())

### Добавим кнопку

In [ ]:
button = widgets.Button(description='Click me')
output = widgets.Output()

display(button, output) 
counter = 0 

def clickme(b):
  with output:
    try:
      global counter
      clear_output()
      print('Вакансии: ', counter)
      display(HTML("<h1>" + str(vacs_df['name'][counter])+ ' в ' + str(vacs_df['employer_name'][counter]) + "</h1>"))
      display(vacs[counter])
      display(HTML('<a href="' + str(vacs_df['alternate_url'][counter]) + '" target = "_blank">Ссылка на вакансию </a>'))
      counter += 1
    except Exception as e:
      print('Вакансии закончились')
button.on_click(clickme)